In [ ]:
from langchain_community.document_loaders import CSVLoader
def load_data(file_path):
    loader = CSVLoader(file_path=file_path, encoding='utf-8')
    documents = loader.load()
    return documents

In [ ]:
documents = load_data("./law_data_3k.csv")

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
def text_split(documents):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500, 
        chunk_overlap=50
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"分割完成！原始文档有 {len(documents)} 条，分割后变成了 {len(split_docs)} 个分块。")
    return split_docs

In [ ]:
split_docs = text_split(documents)

In [ ]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
def build_embeddingsdb(split_docs):
    os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
    model_name = "moka-ai/m3e-base"
    model_kwargs = {'device': 'cpu'} 
    encode_kwargs = {'normalize_embeddings': True} 
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(split_docs, embeddings)
    db.save_local("faiss_legal_db")

In [ ]:
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [ ]:
model_name = "moka-ai/m3e-base"
model_kwargs = {'device': 'cpu'} 
encode_kwargs = {'normalize_embeddings': True}

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from tqdm import tqdm
db = FAISS.from_documents([split_docs[0]], embeddings)
for i in tqdm(range(1, len(split_docs))):
    db.add_documents([split_docs[i]])

In [ ]:
db.save_local("faiss_legal_db")

In [ ]:
query = "借款人去世，继承人是否应履行偿还义务？"

docs = db.similarity_search(query, k=3)

print(f"\n--- 针对问题：'{query}' 的检索结果 ---")
for i, doc in enumerate(docs):
    print(f"\n[结果 {i+1}]:")
    print(doc.page_content) 

In [1]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

model_name = "moka-ai/m3e-base"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

db = FAISS.load_local("faiss_legal_db", embeddings, allow_dangerous_deserialization=True)

d:\Anaconda\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.prompts import ChatPromptTemplate
template = """你是专业的法律知识问答助手。你需要使用以下检索到的上下文片段来回答问题，禁止根据常识和已知信息回答问题。如果你不知道答案，直接回答“未找到相关答案”。

Question: {question} 

Context: {context} 

Answer:

"""

# 2. 生成 ChatPromptTemplate 对象
prompt = ChatPromptTemplate.from_template(template)

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='你是专业的法律知识问答助手。你需要使用以下检索到的上下文片段来回答问题，禁止根据常识和已知信息回答问题。如果你不知道答案，直接回答“未找到相关答案”。\n\nQuestion: {question} \n\nContext: {context} \n\nAnswer:\n\n'), additional_kwargs={})])

In [3]:
import os
from langchain_community.llms import Tongyi

# 3. 设置你的 API Key (请替换为你自己的 KEY)
os.environ["DASHSCOPE_API_KEY"] = "sk-58c6951502944436a98680d9fd90974e"

# 4. 初始化通义千问模型
llm = Tongyi()

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 5. 将 FAISS 数据库转换为检索器 (Retriever)
# search_kwargs={"k": 3} 表示每次找最相关的 3 条法律
retriever = db.as_retriever(
    # search_kwargs={"k": 3}
)

# 6. 构建完整的 RAG 链
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

RAG 系统流水线构建完成！现在可以开始问答了。


In [5]:
question = "借款人去世，继承人是否应履行偿还义务？"
rag_chain.invoke(question)

'根据《中华人民共和国继承法》第三十三条的规定，继承人继承遗产的，应当清偿被继承人依法应当缴纳的税款和债务，但清偿债务以遗产的实际价值为限。如果继承人放弃继承，则对被继承人的债务可以不负偿还责任。因此，继承人是否应履行偿还义务取决于其是否继承遗产以及继承的范围。\n\n答案：继承人应在继承遗产的实际价值范围内承担偿还义务；若放弃继承，可不承担偿还责任。'

In [6]:

test_questions = [
    "借款人去世，继承人是否应履行偿还义务？",
    "如何通过法律手段应对民间借贷纠纷？",
    "没有赡养老人就无法继承财产吗？",
    "谁可以申请撤销监护人的监护资格？",
    "你现在是一个精通中国法律的法官，请对以下案件做出分析：经审理查明：被告人 xxx 于 2017 年 12 月，多次在本市 xxx 盗窃财物。具体事实如下：（一）2017 年 12 月 9 日 15 时许，被告人 xxx 在 xxx 店内，盗窃白色毛衣一件（价值人民币 259 元）。现赃物已起获并发还。（二）2017 年 12 月 9 日 16 时许，被告人 xx 在本市 xxx 店内，盗窃米白色大衣一件（价值人民币 1199 元）。现赃物已起获并发还。（三）2017 年 12 月 11 日 19 时许，被告人 xxx 在本市 xxx 内，盗窃耳机、手套、化妆镜等商品共八件（共计价值人民币 357.3 元）。现赃物已起获并发还。（四）2017 年 12 月 11 日 20 时许，被告人 xx 在本市 xxxx 内，盗窃橙汁、牛肉干等商品共四件（共计价值人民币 58.39 元）。现赃物已起获并发还。2017 年 12 月 11 日，被告人 xx 被公安机关抓获，其到案后如实供述了上述犯罪事实。经鉴定，被告人 xxx 被诊断为精神分裂症，限制刑事责任能力，有受审能力。",
    "你现在是一个精通中国法律的法官，请对以下案件做出分析：2012 年 5 月 1 日，原告 xxx 在被告 xxxx 购买“玉兔牌”香肠 15 包，其中价值 558.6 元的 14 包香肠已过保质期。xxx 到收银台结账后，即径直到服务台索赔，后因协商未果诉至法院，要求 xxxx 店支付 14 包香肠售价十倍的赔偿金 5586 元。"
]

print("开始执行必做任务：6 个问题的输出展示与对比...\n")

for i, query in enumerate(test_questions):
    print(f"{'='*30} 问题 {i+1} {'='*30}")
    print(f"query: {query}\n")

    # 普通检索
    search_results = db.similarity_search(query, k=1) # 拿最相关的一条
    print(f"普通检索:")
    print(f"--- {search_results[0].page_content}...\n")

    # RAG
    rag_answer = rag_chain.invoke(query)
    print(f"RAG:")
    print(f"--- {rag_answer}\n")

    # LLM w/o RAG
    pure_answer = llm.invoke(f"请回答这个法律问题：{query}")
    print(f"LLM w/o RAG:")
    print(f"--- {pure_answer}\n")
    
    print("-" * 70)

开始执行必做任务：6 个问题的输出展示与对比...

============================== 问题 1 ==============================
query: 借款人去世，继承人是否应履行偿还义务？

普通检索:
--- data: 如果人死了，欠下的债家人要承担吗
债务人死亡之后由继承其遗产的人一并继承但是如果没有人继承其遗产或是其没有遗产的话债务随债务人死亡消除债权人无处主张...

RAG:
--- 根据《中华人民共和国继承法》第三十三条的规定，继承人继承遗产的，应当清偿被继承人依法应当缴纳的税款和债务，但清偿债务以遗产的实际价值为限。如果继承人放弃继承，则对被继承人的税款和债务可以不负偿还责任。因此，继承人是否应履行偿还义务取决于其是否继承遗产以及继承的遗产价值。

答案：继承人应在继承遗产的实际价值范围内承担偿还义务；若放弃继承，则不承担偿还责任。

LLM w/o RAG:
--- 根据中国《民法典》的相关规定，借款人去世后，其继承人是否需要履行偿还义务，取决于以下几个法律原则：

1. **债务不因死亡而消灭**  
   借款人死亡并不导致债务自动消灭。根据《中华人民共和国民法典》第1159条和第1161条的规定，被继承人生前所负的合法债务，应当由其遗产进行清偿。

2. **“限定继承”原则**  
   继承人仅在继承遗产的实际价值范围内承担清偿责任。也就是说：
   - 如果继承人继承了借款人的遗产，则应在所继承遗产的价值范围内偿还债务；
   - 若遗产不足以清偿全部债务，超出部分继承人没有义务偿还（除非自愿）；
   - 如果继承人放弃继承，则对被继承人的债务不负清偿责任。

3. **清偿顺序**  
   根据《民法典》第1159条，遗产应当优先用于清偿被继承人依法应当缴纳的税款和债务。

4. **继承人的选择权**  
   继承人有权选择接受继承或放弃继承：
   - 接受继承 → 在遗产范围内承担债务；
   - 放弃继承 → 不承担债务，但也不能取得遗产。

✅ 举例说明：  
若借款人去世时留下50万元遗产，同时欠银行40万元债务，则继承人可继承遗产，但需先用遗产偿还40万元债务，剩余10万元方可分配。  
若债务为60万元，遗产仅50万元，则只需偿还50万元，剩余10万元债务无需继